[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/USeebi/jetsim-fork/blob/master/JETSIM_road_following_colab.ipynb)

Jetsim : Road Following
===

### Step 5 from README.md

---

Right click and open 'this' notebook **JESTIM_interactive_regression.ipyn** in Google colab.

Open 'Edit/Notebook settings' toolbar in Google Colab and select **GPU**

If you get any runtime errors at anytime, go to 'Runtime' toolbar and click 'Restart Runtime'. Rerun all cells.

---

In [2]:
#initialize google colab
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
print("DONE")


Mounted at /content/gdrive
DONE


Connect to Donkeycar simulator:
===

NOTE: This notebook is running on a Google GPU so it is not local to your **Donkeycar Simulator** network  

In [ ]:
# Download and install cloudflared
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb && dpkg -i cloudflared-linux-amd64.deb
!cloudflared -v

Selecting previously unselected package cloudflared.
(Reading database ... 125080 files and directories currently installed.)
Preparing to unpack cloudflared-linux-amd64.deb ...
Unpacking cloudflared (2025.10.1) ...
Setting up cloudflared (2025.10.1) ...
Processing triggers for man-db (2.10.2-1) ...
cloudflared version 2025.10.1 (built 2025-10-30-18:35 UTC)


### Change SUBDOMAIN to a host domain name before proceed.

In [ ]:
import subprocess

process = subprocess.Popen(
    [
        'bash', '-c',
        'cloudflared access tcp --hostname <SUBDOMAIN>.trycloudflare.com --url 127.0.0.1:9091 &'
    ],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True  # Use text=True for string output
)

print(f"Process started with PID: {process.pid}")
#stdout_data, stderr_data = process.communicate()

Process started with PID: 2064


Git clone gym-donkeycar to Colab
===

### Add 'jestim-google-colab' dirctory


In [ ]:
# https://github.com/Triton-AI/jetsim
!pip install docopt num2words
!pip install git+https://github.com/tawnkramer/gym-donkeycar.git

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.5/163.5 kB 10.6 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=952773b9e7376d79590d78c9387b80724a7ba64e93a8206e96891e1373f78d04
  Stored in directory: /root/.cache/pip/wheels/1a/bf/a1/4cee4f7678c68c5875ca89eaccf460593539805c3906722228
Successfully built docopt
  Cloning https://github.com/tawnkramer/gym-donkeycar.git to /tmp/pip-req-build-xjpvfe4l
  Running command git clone --filter=blob:none --quiet https://github.com/tawnkramer/gym-donkeycar.git /tmp/pip-req-build-xjpvfe4l
  Resolved https://github.com/tawnkramer/gym-donkeycar.git to commit d6264fa46a93a50f6add282972efd8b70a18489e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 631.1/631.1 kB 15.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Ge

In [ ]:
!git clone https://github.com/USeeBI/jetsim-fork.git

Cloning into 'jetsim-fork'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 75 (delta 37), reused 20 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (75/75), 655.84 KiB | 14.26 MiB/s, done.
Resolving deltas: 100% (37/37), done.


Import and load the model
===


In [1]:
# https://colab.research.google.com/notebooks/gpu.ipynb

import torch
import tensorflow as tf

!nvcc --version
print(torch.__version__, torch.cuda.get_device_name(0))
print(tf.__version__, tf.test.gpu_device_name())

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
2.8.0+cu126 Tesla T4
2.19.0 /device:GPU:0


Load either TensorFlow Keras .h5 or PyTorch .pth model

In [5]:

from tensorflow.python.keras.models import load_model
import tensorflow.python.keras.backend as K
import tensorflow as tf
from os import path

# https://www.codegenes.net/blog/cuda_visible_devices-1-python-pytorch-jupyter-notebook/
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
model_path = path.abspath('./jetsim-fork/jetsim_googlecolab/models/example_model.h5')

# https://www.geeksforgeeks.org/deep-learning/tf-keras-models-load_model-in-tensorflow/
model = load_model(model_path, compile=False)
model.summary()
# sets the global learning phase flag to 0, which is the inference phase
# https://stackoverflow.com/questions/69079614/what-does-tf-keras-backend-set-learning-phase-actually-do
K.set_learning_phase(0)

TF_Model = True; PT_Model = False
camera_w = 160; camera_h = 120
print("DONE")
print(tf.config.list_physical_devices('GPU'))

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
img_in (InputLayer)             [(None, 120, 160, 3) 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 58, 78, 24)   1824        img_in[0][0]                     
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 58, 78, 24)   0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 27, 37, 32)   19232       dropout_1[0][0]                  
____________________________________________________________________________________________

/usr/local/lib/python3.12/dist-packages/tensorflow/python/keras/backend.py:429: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


Load either TensorFlow Keras .h5 or PyTorch .pth model

In [6]:
import torch
import torchvision
from torchsummary import summary

CATEGORIES = ['m1_images']
device = torch.device('cuda')
model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2 * len(CATEGORIES))
model = model.cuda().eval().half()

# Make sure the model you select is in the 'models' folder within this Google drive setup.
model.load_state_dict(torch.load('./gdrive/MyDrive/Colab Notebooks/jetsim_googlecolab/models/3029.pth'))

loaded_model = model.to(device).float()
summary(loaded_model, input_size = (3, 224, 224), batch_size = -1)
loaded_model = model.to(device).half()

PT_Model = True; TF_Model = False
camera_w = 224; camera_h = 224
print("DONE")
torch.cuda.is_available()

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

True

Create the racecar dictionary
===

---

customize your car settings.

Select track to race on. Warren is the default.

---

In [ ]:
#initialize gym wrapper

GYM_DICT={

 'car':{
    #custom settings
    'racer_name': 'Jetsim_NAME',
    'bio' : 'Triton_AI_Jetsim = Jetracer',
    'country' : 'US',
    "guid": "GO_TRITON_AI",
    'body_style' : 'f1',            #car01 | f1 | donkey
    'body_rgb': (24, 43, 200),      #(R, G, B)
    'car_name' : 'Training_Jetsim', #displays above car
    'font_size' : 30,
    #camera settings
    "fov" : 80,
    "fish_eye_x" : 0.0,
    "fish_eye_y" : 0.0,
    "img_w" : camera_w,
    "img_h" : camera_h,
    "img_d" : 3,
    "img_enc" : 'JPG',
    "offset_x" : 0.0, #sides
    "offset_y" : 2.0, #height #Jetsim 2.0
    "offset_z" : 0.0, #forward
    "rot_x" : 20.0,   #tilt #Jetsim 0.0
    #"rot_y": 180,    #rotate
    },

# https://github.com/araffin/gym-donkeycar-1
# warehouse, generated-roads, avc-sparkfun, generated, roboracingleague, waveshare,
# warren, thunderhill, circuit-launch

    'default_connection': 'local',

    'local_connection':{
    'scene_name': 'generated_track', # warren | thunderhill | mini_monaco | roboracingleague_1 | generated_track | generated_road | warehouse | sparkfun_avc | waveshare
######YOUR HOST COMPUTER IP ADDRESS######
    'host': '127.0.0.1', #UPDATE WITH COMPUTER IP ADDRESS HOSTING SIM
######YOUR HOST COMPUTER IP ADDRESS######
    'port': 9091,
    'artificial_latency': 0},

    'remote_connection':{
        'scene_name': 'warehouse',
###### REMOTE IP ADDRESS ######
        'host': '49.131.19.135', # DIY Robocar virtual track server = 'donkey-sim.roboticist.dev'
###### REMOTE IP ADDRESS ######
        'port': 9091,
        'artificial_latency': 0}, # Besides the ping to the remote simulator, how many MORE delay would yo

    'lidar':{
        'enabled': False,
        'deg_inc': 1, # Degree increment between each ray of the lidar
        'max_range': 30.0}, # Max range of the lidar laser
    }


Send dictionary to Triton Racer Gym interface
===

### Make sure the **DIY Robocar server or Donkeycar Simulator** is ONLINE before running the code below.


In [ ]:
import time
%cd jetsim-fork/jetsim_local/resources
from gyminterface import GymInterface
%cd ../../../

# Initialization of the GymInterface (inherited from SDClient) sets up a
# background thread to manage socket communication with the Donkeycar simulator.
gym = GymInterface(gym_config=GYM_DICT)
time.sleep(1)
imageSeen, x, y, z, speed, cte, lidar = gym.step(0.0, 0.0, 0.0, False)
# https://blog.csdn.net/Karen_Yu_/article/details/115293733
#print(imageSeen.shape)
newArray = imageSeen[:]
print(newArray.shape)

#If you run this cell more than once it will generate more cars in the sim. Turn simulator application off and back on to delete all cars.


/content/jetsim-fork/jetsim_local/resources
/content
Loading scene: waveshare
Sending configs...
Sending racer info
Car loaded.
Sending car config
Gym Interface: Camera resolution (224, 224).
(224, 224, 3)


---

This will initialize the **racecar in the simulator**. Check if the car **populated** in the sim.

When you call gym.step(...) the following happens internally:
- A control command is sent to the simulator.
- Your code waits for the next telemetry or relevant response, which is received and processed by on_msg_recv().
- on_msg_recv() updates the internal state, making the latest simulator data available for the next step or observation.

---


In [ ]:
#If the custom car setting did not load correctly, uncomment this to take car out of sim and then re-run the cell above.

#gym.onShutdown()

Run the model
===

In [ ]:
import cv2
import threading
import ipywidgets
from ipywidgets import Layout, Button, Box
import ipywidgets.widgets as widgets
from IPython.display import display

print_widget = widgets.Output()

state_widget = ipywidgets.ToggleButtons(options=['On', 'Off'], description='Camera', value='On')
prediction_widget = ipywidgets.Image(format='jpeg', width=gym.gym_config['img_w'], height=gym.gym_config['img_h'])

live_execution_widget = ipywidgets.VBox([
    prediction_widget,
    state_widget
])

# https://stackoverflow.com/questions/42452516/how-to-make-widgets-in-jupyter-notebook-scrollable
display(
    widgets.HBox(
        [live_execution_widget,
        widgets.VBox([print_widget], layout=Layout(height='500px', overflow_y='auto'))]
    )
)

In [ ]:
%cd jetsim-fork/jetsim_googlecolab
from utils import preprocess
from utils import bgr8_to_jpeg
%cd ../../
import numpy as np
import datetime
import ipywidgets as widgets
from IPython.display import display
from IPython.display import clear_output


#****************PRIMARY CONTROLS*****************
STEERING_GAIN = 2.0             #Scalar or intesity of steering response, usally matches THROTTLE_GAIN.
THROTTLE_GAIN = 0.3             #Scalar or intesity of throttle, usally matches STEERING_GAIN.
MAX           = 0.9             #Limiter for controlling speed.
#********************OPTIONS**********************
AUTO_RESET    = False            #reset the car to the start line if crash detected.
BOOST         = False           #Inital booster THROTTLE = 1.0 at start of race for 5 seconds.
LIMITER       = False           #THROTTLE = MAX
PRINT         = True           #outputs steering & throttle values
STRAIGHT_AWAY = False           #Straight away for max speed based on x and z coordinate from step function.
STRAIGHT_AWAY_THROTTLE = 1.0    #Straight away max speed.
#*******************INITIAL***********************
STEERING_BIAS = 0.0             #Steering offset.
THROTTLE_BIAS = 0.0             #Throttle offset, creeps forward at minimum speed.
BREAKS        = 0.0             #Final break value.
t0 = datetime.datetime.now()    #for lap time
out = 0                         #counter for clearing steering & throttle outputs
TF_out = 0
speed_last = 0                  #auto reset
mark = 0                        #auto reset
stuck = datetime.datetime.now() #auto reset
#*************************************************



#Main Jetsim model loop
print('JETSIM ACTIVATED')
THROTTLE = 1.0
STEERING = 0.0
try:
    while True:

        #_________________________Boost_______________________
        #Inital booster THROTTLE = 1.0 at start of race for 5 seconds.
        if BOOST:
            image, x, y, z, speed, cte, lidar = gym.step(0, 1.0, 0.0, False);
            print("BOOST")
            time.sleep(5) #boost for 5 seconds
            print("DRIVING")
            BOOST = False
        #_____________________________________________________


        if TF_Model:
          image, x, y, z, speed, cte, lidar = gym.step(STEERING, THROTTLE, BREAKS, False);
          img_arr = image.reshape((1,) + image.shape)
          start_time = time.time()
          st, tr = model(img_arr)
          S = st.numpy()[0][0] * 100
          T = tr.numpy()[0][0] * 100
          # Print raw model outputs and prediction time
          print (st.numpy()[0][0], tr.numpy()[0][0])
          print(f'Prediction time: {time.time() - start_time}')
          # Clear output every 10 iterations for Jupyter notebooks
          if TF_out == 10:
            clear_output()
            TF_out = 0
          TF_out+=1
        elif PT_Model:
          #___________________Image processing__________________
          # Send controls to gym wrapper and returns image, coordinates, etc.
          # Run one timestep of the environment’s dynamics. When end of episode is reached,
          # calling reset() to reset this environment’s state.
          image, x, y, z, speed, cte, lidar = gym.step(STEERING, THROTTLE, BREAKS, False);
          t_image = preprocess(image).half()
          output = model(t_image).detach().cpu().numpy().flatten()
          S = float(output[0]) #-1.0 to 1.0 steering decision.
          T = float(output[1]) ##-1.0 to 1.0 throttle decision.
          #_____________________________________________________

        if(state_widget.value == 'On'):
            ann_x = int(gym.gym_config['img_w'] * (x / 2.0 + 0.5)) # -1 left, +1 right
            ann_y = int(gym.gym_config['img_h'] * (y / 2.0 + 0.5)) # -1 top, +1 bottom
            prediction = image.copy()
            prediction = cv2.circle(prediction, (ann_x, ann_y), 8, (255, 0, 0), 3)
            prediction_widget.value = bgr8_to_jpeg(prediction)


        #_____________Steering & Throttle control_____________
        #optimize steering and throttle value. Set the limit on the top speed.
        STEERING = round(S * STEERING_GAIN + STEERING_BIAS, 2)
        THROTTLE = round(T * THROTTLE_GAIN + THROTTLE_BIAS, 2)
        if LIMITER: #Limiter
            if THROTTLE > MAX: #maximum speed
                THROTTLE = MAX
        #____________________________________________________



        #____________________Straight away_____________________
        #if car is on the straightest path of the track, go max speed. Warren track only because of coordinates.
        if STRAIGHT_AWAY and x <= 45 and x >= 5 and z <= -20 and z >= -30: #coordinates
            THROTTLE = STRAIGHT_AWAY_THROTTLE
            STEERING = STEERING*0.5 #reduce oversteer
        #____________________________________________________



        #____________________Print value_____________________
        #print steering and throttle value to see realtime control values.
        if PRINT: #output value
          with print_widget:
            print("Steering = "+str(STEERING)+"   Throttle = "+str(THROTTLE))
            if out == 100:
                clear_output()
                out = 0
            out+=1
        #____________________________________________________



        #____________________Auto reset______________________
        #reset the car to the start line if crash detected.
        if AUTO_RESET and round(speed,0) == round(speed_last,0):
          mark = datetime.datetime.now()
          check = mark - stuck
          if (str(check) > '0:00:07.000000'):
            print('NOT MOVING')
            stuck = datetime.datetime.now()
            BOOST = False
            gym.step(0.0, 0.0, 100.0, True)
            time.sleep(1)
        else:
          speed_last = speed
          mark = stuck
          stuck = datetime.datetime.now()
        #____________________________________________________



        #___________________Show lap time___________________
        #displays by default. Cannot show steering and throttle output at the same time.
        if x>=29 and x<=30 and z <= -20 and z >= -30:
            time_print = datetime.datetime.now() - t0
            if (str(time_print) > '0:00:01.000000'):
                print (time_print)
            t0 = datetime.datetime.now()
        #____________________________________________________


#####STOP the main loop select "Kernel" in the drop down menu and click "Interrupt Kernel"#####
except KeyboardInterrupt:

    #Resets steering and throttle to 0, applies breaks and resets for 1 second.
    print("!!!STOP!!!")
    gym.step(0.0, 0.0, 100.0, True);
    time.sleep(1)

/content/jetsim-fork/jetsim_googlecolab
/content
JETSIM ACTIVATED
!!!STOP!!!
Resetting car...


In [ ]:
#RESET run this cell multiple times to stop the car if it still running.
gym.step(0.0, 0.0, 100.0, True);

Resetting car...


If finished - YOU MUST Take car out of SERVER!!!
===

In [ ]:
gym.onShutdown()